In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import torchvision

In [28]:
# class Net1(nn.Module):
#     def __init__(self):
#         super(Net1, self).__init__()
#         self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
#         self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
#         self.conv2_drop = nn.Dropout2d()
#         self.fc1 = nn.Linear(320, 50)
#         self.fc2 = nn.Linear(50, 20)

#     def forward(self, x):
#         x = F.relu(F.max_pool2d(self.conv1(x), 2))
#         x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
#         x = x.view(-1, 320)
#         x = F.relu(self.fc1(x))
#         x = F.dropout(x, training=self.training)
#         x = self.fc2(x)
#         return x
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        # self.fc3 = nn.Linear(50, 50)
        self.fc2 = nn.Linear(50, 10)
        self.fc3 = nn.Linear(50, 10)

    def forward(self, x):
        # print(x.shape)
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        # x = self.fc3(x)
        # a = F.relu(self.fc2(x))
        # x = F.relu(self.fc3(x))
        a = self.fc2(x)
        x = self.fc3(x)
        return a, F.softmax(x), F.log_softmax(x)

In [29]:

class RNN(nn.Module):
   
   def __init__(self, input_size=28, hidden_size=256, num_layers=2, num_classes=10, sequence_len=28):
      super(RNN, self).__init__()
      self.sequence_len = sequence_len
      self.input_size = input_size
      self.hidden_size = hidden_size
      self.num_layers = num_layers
      self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
      self.fc1 = nn.Linear(hidden_size*sequence_len, num_classes)
      self.fc2 = nn.Linear(hidden_size*sequence_len, num_classes)
   
   def forward(self, data):
      data = data.squeeze(1)
      data = data.reshape(-1, self.sequence_len, self.input_size)

      h0 = torch.zeros(self.num_layers, data.size(0), self.hidden_size)
      c0 = torch.zeros(self.num_layers, data.size(0), self.hidden_size)
      
      out, _ = self.lstm(data, (h0, c0))
      out = out.reshape(out.shape[0], -1)
      a = self.fc1(out)
      x = self.fc2(out)
      return a, F.softmax(x), F.log_softmax(x)


In [30]:
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        # self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        # self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        # self.conv2_drop = nn.Dropout2d()
        # self.fc1 = nn.Linear(320, 50)
        # self.fc3 = nn.Linear(50, 50)
        self.fc2 = nn.Linear(784, 10)
        self.fc3 = nn.Linear(784, 10)

    def forward(self, x):
        # print(x.shape)
        # x = F.relu(F.max_pool2d(self.conv1(x), 2))
        # x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 784)
        # x = F.relu(self.fc1(x))
        # x = F.dropout(x, training=self.training)
        # x = self.fc3(x)
        # a = F.relu(self.fc2(x))
        a = self.fc2(x)
        x = self.fc3(x)
        return a, F.softmax(x), F.log_softmax(x)

In [31]:
# /from torch._C import float64
class Net3(nn.Module):
    def __init__(self, net1, net2):
        super(Net3, self).__init__()
        self.net1 = net1
        self.net2 = net2
        self.rnn = RNN()
        self.softmax = nn.Softmax()
        self.fc1 = nn.Linear(10, 1)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(2, 2)

    def forward(self, x):
        a1, x1, _ = self.net1(x)
        a2, x2, _ = self.net2(x)
        # a2, x2 = self.rnn(x)
        # print(x1.shape)
        # x1 = x1.reshape(64, 10, 2).permute(2,0,1)
        # x2 = x2.reshape(64, 10, 2).permute(2,0,1)

        # a1 = self.fc1(x1[1])
        # a2 = self.fc1(x2[1])

        a1 = self.fc1(a1)
        a2 = self.fc1(a2)

        a = torch.cat((a1, a2), 1)
        # a = torch.tensor([a1, a2]).view(1,-1)
        
        # a = self.fc3(a)
        # a = F.sigmoid(a)
        # a = a / torch.sum(a, axis=1).view(-1,1)
        # print(a.shape)
        a = self.softmax(a)
        # print(a.shape)
        print(a.mean(axis=0))
        
        # y1 = a[:,0].view(-1,1)*x1
        # y2 = a[:,1].view(-1,1)*x2

        y1 = a[:,0].view(-1,1)*x1
        y2 = a[:,1].view(-1,1)*x2

        y = y1+y2

      
        # y = self.fc2(y)

        return F.log_softmax(y)

In [32]:
n_epochs = 10
batch_size_train = 64
batch_size_test = 64
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True, drop_last=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True, drop_last=True)

In [33]:
n_epochs = 2
network1 = Net1()
optimizer = optim.SGD(network1.parameters(), lr=learning_rate,
                      momentum=momentum)


train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

def train(epoch):
  network1.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    _, _, output = network1(data)
    loss = F.nll_loss(output, target)
    # loss += 
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

def test():
  network1.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      _, _, output = network1(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))
  
test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.3044, Accuracy: 975/10000 (10%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.335918
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.303943
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.297740
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.269550
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.177119
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.213631
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.235065
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.075333
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.089083
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.110376
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.943330
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.840787
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.825393
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.457872
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.606867
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.511271
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.204450
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.209653
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.198938
Train Epoch: 1 [12160/

In [34]:
n_epochs = 2
network2 = RNN()
optimizer = optim.SGD(network2.parameters(), lr=learning_rate,
                      momentum=momentum)


train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

def train(epoch):
  network2.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    _, _, output = network2(data)
    loss = F.nll_loss(output, target)
    # loss += 
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

def test():
  network2.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      _, _, output = network2(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))
  
test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.2988, Accuracy: 1157/10000 (12%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.298121
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.295673
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.278807
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.269218
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.256472
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.253178
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.222374
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.218071
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.217779
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.191818
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.183324
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.133142
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.107097
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.092086
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.049209
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.047361
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.940652
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.903634
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.922878
Train Epoch: 1 [12160

KeyboardInterrupt: ignored

In [35]:
train_losses = []
train_counter = []
test_losses = []
n_epochs = 10
learning_rate = 0.001
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]
# network1 = Net1()
# network2 = Net2()
network = Net3(network1, network2)

for name, param in network.named_parameters():
  if name != "fc1.weight" and name != "fc1.bias" and name != "fc2.weight" and name != "fc2.bias" and name != "fc3.weight" and name != "fc3.bias":
    print(name)
    param.required_grad = False

optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)


def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    # loss += 
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
      
def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

net1.conv1.weight
net1.conv1.bias
net1.conv2.weight
net1.conv2.bias
net1.fc1.weight
net1.fc1.bias
net1.fc2.weight
net1.fc2.bias
net1.fc3.weight
net1.fc3.bias
net2.lstm.weight_ih_l0
net2.lstm.weight_hh_l0
net2.lstm.bias_ih_l0
net2.lstm.bias_hh_l0
net2.lstm.weight_ih_l1
net2.lstm.weight_hh_l1
net2.lstm.bias_ih_l1
net2.lstm.bias_hh_l1
net2.fc1.weight
net2.fc1.bias
net2.fc2.weight
net2.fc2.bias
rnn.lstm.weight_ih_l0
rnn.lstm.weight_hh_l0
rnn.lstm.bias_ih_l0
rnn.lstm.bias_hh_l0
rnn.lstm.weight_ih_l1
rnn.lstm.weight_hh_l1
rnn.lstm.bias_ih_l1
rnn.lstm.bias_hh_l1
rnn.fc1.weight
rnn.fc1.bias
rnn.fc2.weight
rnn.fc2.bias
tensor([0.4562, 0.5438])
tensor([0.4164, 0.5836])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4

tensor([0.4383, 0.5617])
tensor([0.4061, 0.5939])
tensor([0.4332, 0.5668])
tensor([0.4332, 0.5668])
tensor([0.4112, 0.5888])
tensor([0.4524, 0.5476])
tensor([0.4555, 0.5445])
tensor([0.4478, 0.5522])
tensor([0.4577, 0.5423])
tensor([0.4454, 0.5546])
tensor([0.4470, 0.5530])
tensor([0.4348, 0.5652])
tensor([0.4173, 0.5827])
tensor([0.4220, 0.5780])
tensor([0.4431, 0.5569])
tensor([0.3940, 0.6060])
tensor([0.4229, 0.5771])
tensor([0.4330, 0.5670])
tensor([0.4698, 0.5302])
tensor([0.4683, 0.5317])
tensor([0.4433, 0.5567])
tensor([0.4361, 0.5639])
tensor([0.4308, 0.5692])
tensor([0.4489, 0.5511])
tensor([0.3957, 0.6043])
tensor([0.4219, 0.5781])
tensor([0.4249, 0.5751])
tensor([0.4048, 0.5952])
tensor([0.4677, 0.5323])
tensor([0.4839, 0.5161])
tensor([0.4784, 0.5216])
tensor([0.4215, 0.5785])
tensor([0.4700, 0.5300])
tensor([0.4398, 0.5602])
tensor([0.4445, 0.5555])
tensor([0.4465, 0.5535])
tensor([0.4372, 0.5628])
tensor([0.4480, 0.5520])
tensor([0.4676, 0.5324])
tensor([0.4596, 0.5404])


KeyboardInterrupt: ignored

In [ ]:
from torchsummary import summary

# summary(network, (1, 28, 28))


  # print(name, param.shape)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Streaming output truncated to the last 5000 lines.
tensor([0.5141, 0.4859])
tensor([0.5163, 0.4837])
tensor([0.5124, 0.4876])
tensor([0.5083, 0.4917])
tensor([0.5070, 0.4930])
tensor([0.5197, 0.4803])
tensor([0.5086, 0.4914])
tensor([0.5115, 0.4885])
Train Epoch: 6 [55680/60000 (93%)]	Loss: 0.201270
tensor([0.5204, 0.4796])
tensor([0.5320, 0.4680])
tensor([0.5101, 0.4899])
tensor([0.5141, 0.4859])
tensor([0.5118, 0.4882])
tensor([0.5089, 0.4911])
tensor([0.4999, 0.5001])
tensor([0.5225, 0.4775])
tensor([0.5194, 0.4806])
tensor([0.5059, 0.4941])
Train Epoch: 6 [56320/60000 (94%)]	Loss: 0.246359
tensor([0.5165, 0.4835])
tensor([0.5171, 0.4829])
tensor([0.5218, 0.4782])
tensor([0.5156, 0.4844])
tensor([0.5222, 0.4778])
tensor([0.5222, 0.4778])
tensor([0.5214, 0.4786])
tensor([0.5220, 0.4780])
tensor([0.5198, 0.4802])
tensor([0.5119, 0.4881])
Train Epoch: 6 [56960/60000 (95%)]	Loss: 0.138904
tensor([0.5154, 0.4846])
tensor([0.5084, 0.4916])
tensor([0.5228, 0.4772])
tensor([0.5203, 0.4797])

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)